In [ ]:
# Importing packages neeeded to define the molecule and map the hamiltonian

from qiskit_nature.second_q.drivers import PySCFDriver
from pyscf import scf, gto
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

# First we define a function that takes as input the interatomic distance and returns the mapped hamiltonian, as well as some properties of the problem.

def get_qubit_op(d):

    h2 = MoleculeInfo(["H", "H"], 
                        [(0.0, 0.0, 0.0), (0.0, 0.0, d)], 
                        charge= 0,
                        multiplicity= 1 # 2*spin + 1,
                        )

    driver = PySCFDriver.from_molecule(h2 , basis="sto3g")
    
    problem = driver.run() 
    
    num_particles=problem.num_particles
    
    num_orbitals=int(problem.num_spatial_orbitals)
    
    hamiltonian = problem.hamiltonian
    
    second_q_op = hamiltonian.second_q_op()
    
    mapper = JordanWignerMapper()
    
    converter = QubitConverter(mapper)
    
    qubit_op = converter.convert(second_q_op)

    return qubit_op,problem,converter, num_particles, num_orbitals


# Now a function that will compute de SCF-HF energy for the molecule 

def hf_ener(d):
    
    mol = gto.M(
      verbose=0,
      atom=[['H',(0, 0, 0)], 
            ['H',(0, 0, d)]],
      basis="sto3g",
      charge = 0
    )
    
    mf = scf.RHF(mol).x2c().run()
    hf = mf.kernel()

    return hf


# Now we import packages needed for the VQE calculation

from qiskit.algorithms.optimizers import SLSQP, COBYLA, SPSA
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory, NumPyEigensolverFactory, ExcitedStatesEigensolver, QEOM
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.algorithms.variational_algorithm import VariationalAlgorithm
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
from qiskit.circuit.library import EfficientSU2

import numpy as np

def filter_criterion(eigenstate, eigenvalue, aux_values): # This filter function must be added to get S=1 states.
    
    return np.isclose(aux_values["ParticleNumber"][0], 2.0)


def np_sol(problem, converter):
    
    solver_ground = NumPyMinimumEigensolverFactory()
    
    calc_gs = GroundStateEigensolver(converter, solver_ground)
    
    np_result_gs = calc_gs.solve(problem)
    
    np_result_gs = np_result_gs.total_energies[0].real
    
    
    solver_exc = NumPyEigensolverFactory(filter_criterion = filter_criterion)

    calc_exc = ExcitedStatesEigensolver(converter, solver_exc)
    
    np_result_exc = calc_exc.solve(problem)
    
    np_result_exc = problem.interpret(np_result_exc).total_energies
    
    return np_result_gs, np_result_exc


# estimator=Estimator()

estimator = AerEstimator()

slsqp = SLSQP()

cobyla = COBYLA()

spsa = SPSA()

# Define a function that calculates the variational ground state energy at a given distance.

def vqe_sol(num_orbitals,num_particles,converter,problem):

    
    init_state = HartreeFock(num_orbitals,num_particles, converter)
    
    uccsd = UCCSD(num_orbitals, num_particles, converter,initial_state=init_state) 
    
    two_local = EfficientSU2(num_qubits=4, entanglement='linear', initial_state=init_state)

    
    vqe_slsqp = VQE(estimator, uccsd, slsqp)
    
    vqe_cobyla = VQE(estimator, uccsd, cobyla)
    
    vqe_spsa = VQE(estimator, uccsd, spsa)
    
    vqe_two_local = VQE(estimator, two_local, slsqp)
    
    
    vqe_solution_slsqp = vqe_slsqp.compute_minimum_eigenvalue(qubit_op)
    
    vqe_solution_cobyla = vqe_cobyla.compute_minimum_eigenvalue(qubit_op)
    
    vqe_solution_spsa = vqe_spsa.compute_minimum_eigenvalue(qubit_op)
    
    vqe_solution_two_local = vqe_two_local.compute_minimum_eigenvalue(qubit_op)
    
    
    vqe_result_slsqp = problem.interpret(vqe_solution_slsqp).total_energies[0].real
    
    vqe_result_cobyla = problem.interpret(vqe_solution_cobyla).total_energies[0].real
    
    vqe_result_spsa = problem.interpret(vqe_solution_spsa).total_energies[0].real
    
    vqe_result_two_local = problem.interpret(vqe_solution_two_local).total_energies[0].real

    
    hf_result = hf_ener(d)
    
   
    gse = GroundStateEigensolver(converter, vqe_slsqp) # Excited state calculation
    
    qeom = QEOM ( gse, estimator, "sd")
    
    qeom_solution = qeom.solve(problem)
    
    qeom_result = problem.interpret(qeom_solution).total_energies # problem.interpret().total_energies
    # returns an array where a[0] is gs energy, a[1] is first excited energy and so on
    
    
    return hf_result, qeom_result, vqe_result_slsqp, vqe_result_cobyla, vqe_result_spsa, vqe_result_two_local


# Define the arrays to loop over

distances = np.loadtxt('distances.txt') # Loads data (in a.u.)
distances = distances * 0.529177249 # In Angstrom
ref_ener_gs = np.loadtxt('exact_ener.txt')


vqe_energies = []
hf_energies = []
np_energies = []

qeom_first_exc = []
qeom_second_exc = []
qeom_third_exc = []

np_first_exc = []
np_second_exc = []
np_third_exc = []

vqe_energies_slsqp = []
vqe_energies_cobyla = []
vqe_energies_spsa = []

vqe_energies_two_local = []

error_first_exc = []
error_second_exc = []
error_third_exc = []

error_vqe = []
error_hf = []
error_np = []

error_vqe_np = []
error_hf_np = []
error_ref_np = []

error_slsqp = []
error_cobyla = []
error_spsa = []

error_slsqp_np = []
error_cobyla_np = []
error_spsa_np = []

error_two_local = []

error_two_local_np = []

# Start a timer to get the execution time

import timeit

start = timeit.default_timer()
counter=0;

# Main loop.

for d in distances: # For each interatomic distance,
    
    # first we get the mapped hamiltonian, as well as some of its properties

    (qubit_op,problem,converter, num_particles, num_orbitals) = get_qubit_op(d)

    # then we calculate all the different energies (HF, VQE, QEOM),
    
    (hf_result, qeom_result, vqe_result_slsqp, vqe_result_cobyla, vqe_result_spsa, vqe_result_two_local) = vqe_sol(num_orbitals,num_particles,converter,problem)
    
    # as well as the exact diagonalization energy

    (np_result_gs, np_result_exc) = np_sol(problem, converter)
    
    # Now we store all the values obtained in their corresponding lists

    vqe_energies.append(vqe_result_slsqp)  
    
    hf_energies.append(hf_result)
    
    np_energies.append(np_result_gs)
    
    
    qeom_first_exc.append(qeom_result[1])   
    
    qeom_second_exc.append(qeom_result[2]) 
    
    qeom_third_exc.append(qeom_result[3])  
    
    # For excited states energies provided by NumPy, it gets a little trickier since the first three states 
    # 1,2,3 have the same energy (they correspond to total spin S = 1 and m_S= -1,0,1) and the other two have
    # S = 0. In short, from the first three we just need to pick one.
    
    np_first_exc.append(np_result_exc[1])    
    
    np_second_exc.append(np_result_exc[4])  
                                            # These two have S = 0
    np_third_exc.append(np_result_exc[5])
    
    
    vqe_energies_slsqp.append(vqe_result_slsqp)        
        
    vqe_energies_cobyla.append(vqe_result_cobyla)   
    
    vqe_energies_spsa.append(vqe_result_spsa)   
    
    
    vqe_energies_two_local.append(vqe_result_two_local)  
    
    # The print below will help us keep track of the loop

    print(f"Interatomic Distance: {np.round(d, 2)}")

    # Once everything is calculated, we compute the errors in energy for that particular distance
    
    error_vqe.append(abs(ref_ener_gs[counter] - vqe_result_slsqp)) #/exact_ener[counter])
    error_hf.append(abs(ref_ener_gs[counter] - hf_result))
    error_np.append(abs(ref_ener_gs[counter] - np_result_gs))
    
    error_vqe_np.append(abs(np_result_gs - vqe_result_slsqp))
    error_hf_np.append(abs(np_result_gs - hf_result))
    error_ref_np.append(abs(np_result_gs - ref_ener_gs[counter]))
    
    error_first_exc.append(abs(qeom_first_exc[counter] - np_first_exc[counter])) #/exact_ener[counter])
    error_second_exc.append(abs(qeom_second_exc[counter] - np_second_exc[counter]))
    error_third_exc.append(abs(qeom_third_exc[counter] - np_third_exc[counter]))
    
    error_slsqp.append(abs(ref_ener_gs[counter] - vqe_energies_slsqp[counter])) #/exact_ener[counter])
    error_cobyla.append(abs(ref_ener_gs[counter] - vqe_energies_cobyla[counter]))
    error_spsa.append(abs(ref_ener_gs[counter] - vqe_energies_spsa[counter]))
   
    error_slsqp_np.append(abs(np_result_gs - vqe_energies_slsqp[counter])) #/exact_ener[counter])
    error_cobyla_np.append(abs(np_result_gs - vqe_energies_cobyla[counter]))
    error_spsa_np.append(abs(np_result_gs - vqe_energies_spsa[counter]))

    error_two_local.append(abs(ref_ener_gs[counter] - vqe_energies_two_local[counter]))
    error_two_local_np.append(abs(np_result_gs - vqe_energies_two_local[counter]))
    
    # Update the counter

    counter = counter + 1
        

print("All energies have been calculated")

# We stop the timer to get a glimpse of the execution time 

stop = timeit.default_timer()

execution_time = stop - start

print("Program Executed in "+str(execution_time)) # It returns time in seconds

vqe_energies = vqe_energies_slsqp

# Finally we store all the data obtained in a dictionary to plot it later

h2 = {}
h2['distances']= distances
h2['ref_ener_gs']= ref_ener_gs
h2['vqe_energies'] = vqe_energies
h2['hf_energies'] = hf_energies
h2['np_energies'] = np_energies
h2['qeom_first_exc'] = qeom_first_exc
h2['np_first_exc'] = np_first_exc
h2['qeom_second_exc'] = qeom_second_exc
h2['np_second_exc'] = np_second_exc
h2['qeom_third_exc'] = qeom_third_exc
h2['np_third_exc'] = np_third_exc
h2['vqe_energies_slsqp'] = vqe_energies_slsqp
h2['vqe_energies_cobyla'] = vqe_energies_cobyla
h2['vqe_energies_spsa'] = vqe_energies_spsa
h2['vqe_energies_two_local'] = vqe_energies_two_local

np.savez("h2", **h2)